In [2]:
import tensorflow as tf
import numpy

def write_binary():
    writer = tf.python_io.TFRecordWriter('D://tmp//data.tfrecord')

    for i in range(0, 2):
        a = 0.618 + i
        b = [2016 + i, 2017+i]
        c = numpy.array([[0, 1, 2],[3, 4, 5]]) + i
        c = c.astype(numpy.uint8)
        c_raw = c.tostring()

        example = tf.train.Example(
            features=tf.train.Features(
                feature={
                    'a': tf.train.Feature(
                        float_list=tf.train.FloatList(value=[a])
                    ),

                    'b': tf.train.Feature(
                        int64_list=tf.train.Int64List(value=b)
                    ),
                    'c': tf.train.Feature(
                        bytes_list=tf.train.BytesList(value=[c_raw])
                    )
                }
            )
        )
        serialized = example.SerializeToString()
        writer.write(serialized)

    writer.close()

def read_single_sample(filename):
    # output file name string to a queue
    filename_queue = tf.train.string_input_producer([filename], num_epochs=None)

    # create a reader from file queue
    reader = tf.TFRecordReader()
    _, serialized_example = reader.read(filename_queue)

    # get feature from serialized example

    features = tf.parse_single_example(
        serialized_example,
        features={
            'a': tf.FixedLenFeature([], tf.float32),
            'b': tf.FixedLenFeature([2], tf.int64),
            'c': tf.FixedLenFeature([], tf.string)
        }
    )

    a = features['a']

    b = features['b']

    c_raw = features['c']
    c = tf.decode_raw(c_raw, tf.uint8)
    c = tf.reshape(c, [2, 3])

    return a, b, c

#-----main function-----
sess = tf.InteractiveSession()
write_binary()

# create tensor
a, b, c = read_single_sample('d://tmp//data.tfrecord')
print('---------------a:',a)
a_batch, b_batch, c_batch = tf.train.shuffle_batch([a, b, c], batch_size=3, capacity=200, min_after_dequeue=100, num_threads=2)

queues = tf.get_collection(tf.GraphKeys.QUEUE_RUNNERS)

# sess
#sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)
tf.train.start_queue_runners(sess=sess)

print(a.eval())
print(b.eval())
print(c.eval())
a_val, b_val, c_val = sess.run([a_batch, b_batch, c_batch])
print('-----batch1---------')
print(a_val, b_val, c_val)
a_val, b_val, c_val = sess.run([a_batch, b_batch, c_batch])
print('-----batch2---------')
print(a_val, b_val, c_val)

---------------a: Tensor("ParseSingleExample_1/Squeeze_a:0", shape=(), dtype=float32)
1.618
[2016 2017]
[[0 1 2]
 [3 4 5]]
-----batch1---------
[ 0.61799997  0.61799997  0.61799997] [[2016 2017]
 [2016 2017]
 [2016 2017]] [[[0 1 2]
  [3 4 5]]

 [[0 1 2]
  [3 4 5]]

 [[0 1 2]
  [3 4 5]]]
-----batch2---------
[ 0.61799997  1.61800003  0.61799997] [[2016 2017]
 [2017 2018]
 [2016 2017]] [[[0 1 2]
  [3 4 5]]

 [[1 2 3]
  [4 5 6]]

 [[0 1 2]
  [3 4 5]]]


In [1]:
import tensorflow as tf
with tf.name_scope('scope_a'):
    a = tf.add(1,2,name = 'a_add')
    b = tf.multiply(a,3,name='a_mul')
with tf.name_scope('scope_c'):
    c = tf.add(4,5,name='c_add')
    d = tf.multiply(c,6,name='c_mul')
e = tf.add(b,d,name='output')

writer = tf.summary.FileWriter('d://tmp//name_scope_1',graph = tf.get_default_graph())
writer.close()

In [3]:
#----------------------simple cnn
# coding: utf-8

# In[1]:

from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf


# In[2]:

mnist = input_data.read_data_sets("MNIST_data/",one_hot = True)
sess = tf.InteractiveSession()


# In[3]:

#########定义初始化函数###############
def weight_variable(shape): #先定义好初始化函数以便重复使用。我们需要给权重制造一些随机噪声来打破完全对称
    initial = tf.truncated_normal(shape,stddev = 0.1) #返回A tensor of the specified shape filled with random truncated normal values.
    return tf.Variable(initial)  #返回一个Tensor.Returns the Tensor used as the initial value for the variable.

def bias_variable(shape):
    initial = tf.constant(0.1,shape = shape) #返回A Constant Tensor.
    return tf.Variable(initial)


# In[4]:

#########定义神经网络中共的核心函数###############
def conv2d(x,W): #W卷积参数[5,5,1,32] 5*5代表卷积核尺寸；1：代表通道数,灰度图像为1，RGB图像为3；32卷积核数目
    return tf.nn.conv2d(x,W,strides = [1,1,1,1],padding = "SAME") #返回A Tensor. Has the same type as input
def max_pool_2_2(x): #这里使用2*2的最大池化，即将一个2*2变成1*1.因为希望整体缩小图片尺寸，所以strides设置为横竖两个方向以2为步长。
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides = [1,2,2,1],padding = "SAME") #返回A Tensor with type tf.float32. The max pooled output tensor.


# In[5]:

#########在正式设计神经网络之前先定义输入的placeholder函数###########
x = tf.placeholder(tf.float32,[None,784])  #输入样本参量
y_ = tf.placeholder(tf.float32,[None,10])  #输入样本标签,输出A Tensor that may be used as a handle for feeding a value, but not evaluated directly.
x_image = tf.reshape(x,[-1,28,28,1])#-1代表样本数量不确定，每个样本28*28，1代表颜色通道数


# In[6]:

###############定义第一个卷积层#####################################
W_conv1 = weight_variable([5,5,1,32]) #卷积核尺寸5*5,1个颜色通道，32个不同卷积核,感觉像高维向量
b_conv1 = bias_variable([32])
h_conv1 = tf.nn.relu(conv2d(x_image,W_conv1) + b_conv1)
h_pool1 = max_pool_2_2(h_conv1)
###############定义第二个卷积层#####################################
W_conv2 = weight_variable([5,5,32,64]) #卷积核尺寸5*5,注意这里变为32个通道，因为第一层输出32个卷积核结果，64个不同卷积核,感觉像高维向量
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1,W_conv2) + b_conv2)
h_pool2 = max_pool_2_2(h_conv2)
###############最后跟一个全连接层####################
#前面经过两次池化，所以边长只有原来1/4了，图片尺寸为7*7，第二层卷积核数量为64，其输出tensor为7*7*64,隐含节点数为1024。
W_fc1 = weight_variable([7*7*64,1024])  #不知道这里7*7*64怎么来的
b_fc1 = bias_variable([1024])
h_pool2_flat = tf.reshape(h_pool2,[-1,7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat,W_fc1) + b_fc1)
##################加一个Dropout层####################
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1,keep_prob)
##################最后将Dropout层连接一个Softmax层########
W_fc2 = weight_variable([1024,10])
b_fc2 = bias_variable([10])
y_conv = tf.nn.softmax(tf.matmul(h_fc1_drop,W_fc2) + b_fc2)


# In[7]:

###########定义损失函数,########################
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y_conv),reduction_indices=[1]))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
##########评测准确率################################
correct_prediction = tf.equal(tf.argmax(y_conv,1),tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))


# In[8]:

#########################训练################
tf.global_variables_initializer().run()

for i in range(150):
    batch = mnist.train.next_batch(50)
    if i%10 == 0:
        train_accuracy = accuracy.eval(feed_dict = {x:batch[0],y_:batch[1],keep_prob:1.0})
        print("step %d, training accuracy %g" %(i,train_accuracy))
    train_step.run(feed_dict={x:batch[0],y_:batch[1],keep_prob:0.5})
    
print("test accuracy %g" %accuracy.eval(feed_dict = {x:mnist.test.images,y_:mnist.test.labels,keep_prob:1.0}))
#eval参见 tf.Tensor



Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
step 0, training accuracy 0.08
step 10, training accuracy 0.26
step 20, training accuracy 0.54
step 30, training accuracy 0.42
step 40, training accuracy 0.58
step 50, training accuracy 0.58
step 60, training accuracy 0.84
step 70, training accuracy 0.82
step 80, training accuracy 0.76
step 90, training accuracy 0.72
step 100, training accuracy 0.84
step 110, training accuracy 0.84
step 120, training accuracy 0.78
step 130, training accuracy 0.94
step 140, training accuracy 0.76
test accuracy 0.884


In [ ]:
#--------------------------生成logs文件夹（tensorboard）
# coding=utf-8

import tensorflow as tf
"""
首先载入Tensorflow，并设置训练的最大步数为1000,学习率为0.001,dropout的保留比率为0.9。
同时，设置MNIST数据下载地址data_dir和汇总数据的日志存放路径log_dir。
这里的日志路径log_dir非常重要，会存放所有汇总数据供Tensorflow展示。
"""

from tensorflow.examples.tutorials.mnist import input_data
max_step = 1000
learning_rate = 0.001
dropout = 0.9
data_dir = 'MNIST_data/'
log_dir = 'logs/'

# 使用input_data.read_data_sets下载MNIST数据，并创建Tensorflow的默认Session
mnist = input_data.read_data_sets(data_dir, one_hot=True)
sess = tf.InteractiveSession()

"""
为了在TensorBoard中展示节点名称，设计网络时会常使用tf.name_scope限制命名空间，
在这个with下所有的节点都会自动命名为input/xxx这样的格式。
定义输入x和y的placeholder，并将输入的一维数据变形为28×28的图片存储到另一个tensor，
这样就可以使用tf.summary.image将图片数据汇总给TensorBoard展示了。
"""
with tf.name_scope('input'):
    x = tf.placeholder(tf.float32, [None, 784], name='x_input')
    y = tf.placeholder(tf.float32, [None, 10], name='y_input')

with tf.name_scope('input_reshape'):
    image_shaped_input = tf.reshape(x, [-1, 28, 28, 1])
    tf.summary.image('input', image_shaped_input, 10)

# 定义神经网络模型参数的初始化方法，
# 权重依然使用常用的truncated_normal进行初始化，偏置则赋值为0.1
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

# 定义对Variable变量的数据汇总函数
"""
计算出Variable的mean,stddev,max和min，
对这些标量数据使用tf.summary.scalar进行记录和汇总。
同时，使用tf.summary.histogram直接记录变量var的直方图。
"""
def variable_summaries(var):
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean', mean)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var-mean)))
        tf.summary.scalar('stddev', stddev)
        tf.summary.scalar('max', tf.reduce_max(var))
        tf.summary.scalar('min', tf.reduce_min(var))
        tf.summary.histogram('histogram', var)

# 设计一个MLP多层神经网络来训练数据，在每一层中都会对模型参数进行数据汇总。
"""
定一个创建一层神经网络并进行数据汇总的函数nn_layer。
这个函数的输入参数有输入数据input_tensor,输入的维度input_dim,输出的维度output_dim和层名称layer_name，激活函数act则默认使用Relu。
在函数内，显示初始化这层神经网络的权重和偏置，并使用前面定义的variable_summaries对variable进行数据汇总。
然后对输入做矩阵乘法并加上偏置，再将未进行激活的结果使用tf.summary.histogram统计直方图。
同时，在使用激活函数后，再使用tf.summary.histogram统计一次。
"""
def nn_layer(input_tensor, input_dim, output_dim, layer_name,act=tf.nn.relu):
    with tf.name_scope(layer_name):
        with tf.name_scope('weight'):
            weights = weight_variable([input_dim, output_dim])
            variable_summaries(weights)
        with tf.name_scope('biases'):
            biases = bias_variable([output_dim])
            variable_summaries(biases)
        with tf.name_scope('Wx_plus_b'):
            preactivate = tf.matmul(input_tensor, weights) + biases
            tf.summary.histogram('pre_activations', preactivate)
        activations = act(preactivate, name='actvations')
        tf.summary.histogram('activations', activations)
        return activations

"""
使用刚定义好的nn_layer创建一层神经网络，输入维度是图片的尺寸（784=24×24），输出的维度是隐藏节点数500.
再创建一个Droput层，并使用tf.summary.scalar记录keep_prob。然后再使用nn_layer定义神经网络的输出层，激活函数为全等映射，此层暂时不使用softmax,在后面会处理。
"""
hidden1 = nn_layer(x, 784, 500, 'layer1')

with tf.name_scope('dropout'):
    keep_prob = tf.placeholder(tf.float32)
    tf.summary.scalar('dropout_keep_probability', keep_prob)
    dropped = tf.nn.dropout(hidden1, keep_prob)

y1 = nn_layer(dropped, 500, 10, 'layer2', act=tf.identity)

"""
这里使用tf.nn.softmax_cross_entropy_with_logits()对前面输出层的结果进行softmax处理并计算交叉熵损失cross_entropy。
计算平均损失，并使用tf.summary.saclar进行统计汇总。
"""
with tf.name_scope('cross_entropy'):
    diff = tf.nn.softmax_cross_entropy_with_logits(logits=y1, labels=y)
    with tf.name_scope('total'):
        cross_entropy = tf.reduce_mean(diff)
tf.summary.scalar('cross_entropy', cross_entropy)

"""
使用Adma优化器对损失进行优化，同时统计预测正确的样本数并计算正确率accuray，
再使用tf.summary.scalar对accuracy进行统计汇总。
"""
with tf.name_scope('train'):
    train_step = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)
with tf.name_scope('accuracy'):
    with tf.name_scope('correct_prediction'):
        correct_prediction = tf.equal(tf.argmax(y1, 1), tf.arg_max(y, 1))
    with tf.name_scope('accuracy'):
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
tf.summary.scalar('accuracy', accuracy)

"""
由于之前定义了非常多的tf.summary的汇总操作，一一执行这些操作态麻烦，
所以这里使用tf.summary.merger_all()直接获取所有汇总操作，以便后面执行。
然后，定义两个tf.summary.FileWrite(文件记录器)在不同的子目录，分别用来存放训练和测试的日志数据。
同时，将Session的计算图sess.graph加入训练过程的记录器，这样在TensorBoard的GRAPHS窗口中就能展示整个计算图的可视化效果。
最后使用tf.global_variables_initializer().run()初始化全部变量。
"""
merged = tf.summary.merge_all()
train_writer = tf.summary.FileWriter(log_dir + '/train', sess.graph)
test_writer = tf.summary.FileWriter(log_dir + '/test')
tf.global_variables_initializer().run()

"""
定义feed_dict的损失函数。
该函数先判断训练标记，如果训练标记为true,则从mnist.train中获取一个batch的样本，并设置dropout值;
如果训练标记为False，则获取测试数据，并设置keep_prob为1,即等于没有dropout效果。
"""
def feed_dict(train):
    if train:
        xs, ys = mnist.train.next_batch(100)
        k = dropout
    else:
        xs, ys = mnist.test.images, mnist.test.labels
        k = 1.0
    return {x: xs, y: ys, keep_prob: k}

# 实际执行具体的训练，测试及日志记录的操作
"""
首先，使用tf.train.Saver()创建模型的保存器。
然后，进入训练的循环中，每隔10步执行一次merged（数据汇总），accuracy（求测试集上的预测准确率）操作，
并使应test_write.add_summary将汇总结果summary和循环步数i写入日志文件;
同时每隔100步，使用tf.RunOption定义Tensorflow运行选项，其中设置trace_level为FULL——TRACE,
并使用tf.RunMetadata()定义Tensorflow运行的元信息，
这样可以记录训练是运算时间和内存占用等方面的信息.
再执行merged数据汇总操作和train_step训练操作，将汇总summary和训练元信息run_metadata添加到train_writer.
平时，则执行merged操作和train_step操作，并添加summary到trian_writer。
所有训练全部结束后，关闭train_writer和test_writer。
"""
saver = tf.train.Saver()
for i in range(max_step):
    if i % 10 == 0:
        summary, acc = sess.run([merged, accuracy], feed_dict=feed_dict(False))
        test_writer.add_summary(summary, i)
        print('Accuracy at step %s: %s' % (i, acc))
    else:
        if i % 100 == 99:
            run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
            run_metadata = tf.RunMetadata()
            summary, _ = sess.run([merged, train_step], feed_dict=feed_dict(True),
                                  options=run_options, run_metadata=run_metadata)
            train_writer.add_run_metadata(run_metadata, 'step%03d' % i)
            train_writer.add_summary(summary, i)
            saver.save(sess, log_dir+"/model.ckpt", i)
            print('Adding run metadata for', i)
        else:
            summary, _ = sess.run([merged, train_step], feed_dict=feed_dict(True))
            train_writer.add_summary(summary, i)
train_writer.close()
test_writer.close()


In [1]:
#--------------载入模型例1
import tensorflow as tf
#Prepare to feed input, i.e. feed_dict and placeholders
w1 = tf.placeholder("float", name="w1")
w2 = tf.placeholder("float", name="w2")
b1= tf.Variable(2.0,name="bias")
feed_dict ={w1:4,w2:8}

#Define a test operation that we will restore
w3 = tf.add(w1,w2)
w4 = tf.multiply(w3,b1,name="op_to_restore")
sess = tf.Session()
sess.run(tf.global_variables_initializer())

#Create a saver object which will save all the variables
saver = tf.train.Saver()

#Run the operation by feeding input
print(sess.run(w4,feed_dict))

#Prints 24 which is sum of (w1+w2)*b1 
#Now, save the graph
saver.save(sess, './model/model_name',global_step=1000)

24.0


'./model/model_name-1000'

In [1]:
import tensorflow as tf
sess=tf.Session()    
#First let's load meta graph and restore weights
saver = tf.train.import_meta_graph('./model/model_name-1000.meta')
saver.restore(sess,tf.train.latest_checkpoint('./model/'))

#---------------
v = tf.global_variables()
print('载入模型后   v:')
print(v)

# Access saved Variables directly
print(sess.run('bias:0'))
# This will print 2, which is the value of bias that we saved
# Now, let's access and create placeholders variables and
# create feed-dict to feed new data

graph = tf.get_default_graph()
w1 = graph.get_tensor_by_name("w1:0")
w2 = graph.get_tensor_by_name("w2:0")
feed_dict ={w1:13.0,w2:17.0}

#---------------
v = tf.global_variables()
print('定义w1 w2   v:')
print(v)

#Now, access the op that you want to run. 
op_to_restore = graph.get_tensor_by_name("op_to_restore:0")
print(sess.run(op_to_restore,feed_dict))
#This will print 60 which is calculated

INFO:tensorflow:Restoring parameters from ./model/model_name-1000
载入模型后   v:
[<tf.Variable 'bias:0' shape=() dtype=float32_ref>]
2.0
定义w1 w2   v:
[<tf.Variable 'bias:0' shape=() dtype=float32_ref>]
60.0


In [3]:
#-----------------tf.global_variables
import tensorflow as tf
sess = tf.InteractiveSession()
a = tf.Variable(4,tf.float32,name='a')
b = tf.Variable(3,name='b')
tf.global_variables_initializer().run()
print(a.eval())

v = tf.global_variables()
print(v)
#[<tf.Variable 'Variable:0' shape=() dtype=int32_ref>, <tf.Variable 'Variable_1:0' shape=() dtype=int32_ref>]

4
[<tf.Variable 'Variable:0' shape=() dtype=int32_ref>, <tf.Variable 'Variable_1:0' shape=() dtype=int32_ref>, <tf.Variable 'a:0' shape=() dtype=int32_ref>, <tf.Variable 'b:0' shape=() dtype=int32_ref>, <tf.Variable 'a_1:0' shape=() dtype=int32_ref>, <tf.Variable 'b_1:0' shape=() dtype=int32_ref>]


In [3]:
#---------------载入模型例2

#!/usr/bin/env python
# -*- coding: utf-8 -*-

import tensorflow as tf
v1 = tf.Variable(1.1, name="v1")
v2 = tf.Variable(1.2, name="v2")

init = tf.global_variables_initializer()
saver = tf.train.Saver()
with tf.Session() as sess:
    sess.run(init)
    print (v2.eval(sess))
    save_path="./model2/model_name"
    saver.save(sess,save_path,global_step=1000)
    print ("Model stored....")

1.2
Model stored....


In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import tensorflow as tf

v3 = tf.Variable(3.0, name="v1")
v4 = tf.Variable(8.0, name="v2")

saver = tf.train.Saver()

with tf.Session() as sess:

    save_path="./model2/"

    saver.restore(sess,save_path)#tf.train.latest_checkpoint('./model2/')
    v = tf.global_variables()
    print(v)
    print("Model restored.")
    print (sess.run(v3))

INFO:tensorflow:Restoring parameters from ./model2/
[<tf.Variable 'v1:0' shape=() dtype=float32_ref>, <tf.Variable 'v2:0' shape=() dtype=float32_ref>]
Model restored.
1.1


In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf

mnist = input_data.read_data_sets("MNIST_data/",one_hot = True)    #导入数据
sess = tf.InteractiveSession()    #开启一个Session

def weight_variable(shape):    #定义一个权重变量
    initial = tf.truncated_normal(shape,stddev=0.1)    #初始化一个正态分布噪声，标准差为0.1
    return tf.Variable(initial)    

def bias_variable(shape):    #定义一个偏置变量
    initial = tf.constant(0.1,shape=shape)    #初始化一个常量，取值为0.1，用来避免死亡节点
    return tf.Variable(initial)    

def conv2d(x,W):    #定义一个卷积层，输入参数为（输入向量X，权重向量W）
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')    #定义一个二维卷积层，

def max_pool_2x2(x):    #定义一个池化层
        return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')    #调用TF的池化层函数

x = tf.placeholder(tf.float32,[None,784])    #定义输入，条目不限，维度为784
y_ = tf.placeholder(tf.float32,[None,10])    #定义输入，条目不限，类别为10
x_image = tf.reshape(x,[-1,28,28,1])     #对1*785的数据进行reshape，转化为二维数据，-1代表样本不固定，尺寸转化为28*28，通道为1

W_conv1 = weight_variable([5,5,1,32])    #初始化第一个卷积层的权重变量，5*5的核，1个通道，32个不同的卷积核
b_conv1 = bias_variable([32])    #初始化对应于卷积核的偏置
h_conv1 = tf.nn.relu(conv2d(x_image,W_conv1)+b_conv1)    #调用卷积层定义函数，初始化第一个卷积层的激活函数，使用relu激活函数
h_pool1 = max_pool_2x2(h_conv1)    #调用池化层定义函数，初始化第一个卷积层之后的池化层

W_conv2 = weight_variable([5,5,32,64])    #初始化第二个卷积层的权重变量，5*5的核，来自上一层的32个通道，64个不同的卷积核
b_conv2 = bias_variable([64])    #对应于64个核的偏置
h_conv2 = tf.nn.relu(conv2d(h_pool1,W_conv2)+b_conv2)   
h_pool2 = max_pool_2x2(h_conv2)

W_fc1 = weight_variable([7*7*64,1024])    #初始化第一个全连接层，大小变为7*7*64，共有1024个隐含节点
b_fc1 = bias_variable([1024])
h_pool2_flat = tf.reshape(h_pool2,[-1,7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat,W_fc1)+b_fc1)

keep_prob = tf.placeholder(tf.float32)    #定义dropout的placeholder
h_fc1_drop = tf.nn.dropout(h_fc1,keep_prob)    #定义dropout层，输入是fc1层输出结果以及keep_prob

W_fc2 = weight_variable([1024,10])    
b_fc2 = bias_variable([10])
y_conv = tf.nn.softmax(tf.matmul(h_fc1_drop,W_fc2)+b_fc2)    #上一层dropout的输出连接一个softmax层


cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_*tf.log(y_conv),reduction_indices=[1]))    #求交叉熵
tf.summary.scalar('cross_entropy_name',cross_entropy)

train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)    #使用Adm优化器求得最小的交叉熵

correct_prediction = tf.equal(tf.argmax(y_conv,1),tf.argmax(y_,1))    #比较预测结果与真实标签准确率

accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))    #求准确率

#-----------summary.scalar函数生成summary protobuf
tf.summary.scalar('accuracy_name',accuracy)
#tf.summary.scalar('cross_entropy',cross_entropy)


#-----------生成saver
saver = tf.train.Saver()
#--------------- 3.定义一个summury op, 用来汇总由scalar_summary记录的所有变量
#merged_summary_op = tf.merge_all_summaries()
merged_summary_op = tf.summary.merge_all()

#-------------- 4.生成一个summary writer对象，需要指定写入路径,例如我这边就是/tmp/logdir
summary_writer = tf.summary.FileWriter('./logdir', sess.graph)

tf.global_variables_initializer().run()
for i in range(1000):    #进行20000次训练迭代
    batch = mnist.train.next_batch(50)    #使用大小为50的mini-batch
    if i%50==0:    #每迭代100次打印一次
        #train_accuracy = accuracy.eval(feed_dict={x:batch[0],y_:batch[1],keep_prob:1.0})     #评测时，dropout比例为1.0
        train_accuracy,summary_str = sess.run([accuracy, merged_summary_op],feed_dict={x:batch[0],y_:batch[1],keep_prob:1.0})
        
        summary_writer.add_summary(summary_str,i)
        print("step %d ,training accuracy %g" %(i,train_accuracy))
        
    train_step.run(feed_dict={x:batch[0],y_:batch[1],keep_prob:0.5})    #训练时，dropout比例为0.5



#------------训练完成后，保存模型
saver.save(sess,'./model_simple_cnn/model_name')

print("test accuracy %g" %accuracy.eval(feed_dict={x:mnist.test.images,y_:mnist.test.labels,keep_prob:1.0}))    #评测时，dropout比例为1.0

summary_writer.close()